In [ ]:
 !nvidia-smi

Mon Mar 14 00:47:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!cp -a ./drive/MyDrive/inpainting_fianl_project/code/. .

In [ ]:
!unzip deep.zip -d .

In [ ]:
!mkdir dataset
# only photos original
# !cp -a ./dataset_photos/photo ./dataset
# all photos
# !cp -a ./dataset_photos/. ./dataset
# only paintings original
# !cp -a ./dataset_paintings/monet ./dataset
# all paintings
!cp -a ./dataset_paintings/. ./dataset

In [ ]:
import torch
dtype = torch.float32
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    torch.cuda.empty_cache()
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda:0


In [ ]:
import os
def get_latest_model_dir():
    dirs = os.listdir('./drive/MyDrive/deep_ella/models_monet/')
    n = 0
    for dir in dirs:
      if int(dir) > n:
        n = int(dir)
    return './drive/MyDrive/deep_ella/models_monet/' + str(n) + '/.', n

models_dir, model_version = get_latest_model_dir()
print(models_dir, model_version)
!cp -a {models_dir} .

./drive/MyDrive/deep_ella/models_monet/780/. 780


In [ ]:
import torchvision.transforms as T
import torchvision.datasets as dset
from torch.utils.data import DataLoader

from options import Options as opt
from inpainting_model import InpaintingModel
from utils import initialize_weights
from masks_generator import random_batch_masks

In [ ]:
def load_data(transform: T.Compose = None, input_size: int = 256):
    if transform is None:
        transform = T.Compose([
            T.Resize(input_size),
            T.CenterCrop(input_size),
            T.ToTensor(),
            T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])

    dataset = dset.ImageFolder(root=opt.dataset_root, transform=transform)
    loader = DataLoader(dataset, batch_size=opt.batch_size,
                        shuffle=True, num_workers=int(opt.workers))

    return loader


def train(train_loader, model: InpaintingModel, epochs=100, print_every=10, save_every=10,
          lr_scheduler_patience=200, lr_scheduler_patience_factor=0.99):
    d_lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=model.optimizer_D, mode='min',
                                                                patience=lr_scheduler_patience,
                                                                factor=lr_scheduler_patience_factor,
                                                                verbose=True)
    g_lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=model.optimizer_G, mode='min',
                                                                patience=lr_scheduler_patience,
                                                                factor=lr_scheduler_patience_factor,
                                                                verbose=True)

    for i in range(epochs):
        for j, (images, _) in enumerate(train_loader):
            masks = random_batch_masks(image_size=images.shape[2], batch_size=images.shape[0], device=model.device)
            model.set_input(images, masks)
            model.optimize_parameters()

            if print_every and not j % print_every:
                print(f"Epoch: {i}, Iteration: {j}, G_Loss: {model.loss_G:.3f}, D_Loss_real: {model.loss_D_real:.3f}, , D_Loss_fake: {model.loss_D_fake:.3f}, Style_Loss: {model.loss_style:.3f}, TV_Loss: {model.loss_tv:.3f}")

            with torch.no_grad():
                d_lr_scheduler.step(model.loss_D)
                g_lr_scheduler.step(model.loss_G)
        if not i % save_every and i != 0:
            ver = model_version + i
            torch.save(model.net_G.state_dict(), f'./best_g_model.pth')
            torch.save(model.net_D.state_dict(), f'./best_d_model.pth')
            !mkdir ./drive/MyDrive/deep_ella/models_monet/{ver}
            !cp best_g_model.pth ./drive/MyDrive/deep_ella/models_monet/{ver}
            !cp best_d_model.pth ./drive/MyDrive/deep_ella/models_monet/{ver}

            print("-------------------------saved models-------------------------")



In [ ]:
def run_model(load_generator=False, load_discriminator=False, epochs: int = 300,
              device: torch.device = torch.device('cpu'), dtype: torch.dtype = torch.float32):
    dataloader = load_data(input_size=128)

    model = InpaintingModel(is_train=True, device=device, dtype = dtype)
    model.net_G.apply(initialize_weights)
    model.net_D.apply(initialize_weights)

    if load_generator:
        model.net_G.load_state_dict(torch.load('./best_g_model.pth', map_location=device))
    if load_discriminator:
        model.net_D.load_state_dict(torch.load('./best_d_model.pth', map_location=device))

    train(dataloader, model, epochs=epochs)

In [ ]:
run_model(load_generator=True, load_discriminator=True, epochs = 300, device=device, dtype=dtype)